# Employee Sentiment Analysis (LLM-based)
Uses HuggingFace DistilBERT model for sentiment classification, then computes monthly scores, rankings, flight risk, and a simple regression.

In [ ]:

import os, sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

PROJECT_ROOT = Path("..").resolve().parent if (Path.cwd().name=="notebooks") else Path(".").resolve()
DATA_XLSX = PROJECT_ROOT/"data"/"test.xlsx"
DATA_CSV = PROJECT_ROOT/"data"/"test.csv"
VIZ_DIR = PROJECT_ROOT/"visualization"
VIZ_DIR.mkdir(parents=True, exist_ok=True)

sys.path.append(str(PROJECT_ROOT/"src"))
from utils import normalize_dataframe, add_sentiment, monthly_scores, top3_by_month, flight_risk, gen_features_for_regression


## Load data

In [ ]:

if DATA_XLSX.exists():
    raw=pd.read_excel(DATA_XLSX)
elif DATA_CSV.exists():
    raw=pd.read_csv(DATA_CSV)
else:
    # synthetic data for demo
    rng=np.random.default_rng(42)
    employees=[f"EMP_{i:03d}" for i in range(1,16)]
    start=pd.Timestamp("2024-01-01")
    rows=[]
    for day in range(120):
        for _ in range(rng.integers(0,3)):
            emp=rng.choice(employees)
            ts=start+pd.Timedelta(days=day)+pd.Timedelta(hours=int(rng.integers(0,24)))
            msg=rng.choice(["Great work on the release!","Blocked by an issue and frustrated","Meeting at 3 PM","I love the team vibe","Unhappy with workload","The build failed again","Happy with the promotion news :)"])
            rows.append({"employee":emp,"message":msg,"timestamp":ts})
    raw=pd.DataFrame(rows)
    print("No file found, using synthetic data.")
raw.head()


## Normalize + LLM Sentiment

In [ ]:

df=normalize_dataframe(raw)
df=add_sentiment(df)
display(df.head())
print("\nLabel counts:\n", df["sentiment_label"].value_counts())


## EDA & Visualizations

In [ ]:

# 1) Sentiment distribution
ax=df["sentiment_label"].value_counts().plot(kind="bar", title="Sentiment Distribution")
plt.tight_layout(); plt.savefig(VIZ_DIR/"sentiment_distribution.png"); plt.show()

# 2) Messages per day
daily=df.set_index("timestamp")["message"].resample("D").count()
daily.plot(title="Messages per Day")
plt.tight_layout(); plt.savefig(VIZ_DIR/"messages_per_day.png"); plt.show()

# 3) Average sentiment score (daily)
score_daily=df.set_index("timestamp")["sentiment_score"].resample("D").mean()
score_daily.plot(title="Average Sentiment Score (Daily)")
plt.tight_layout(); plt.savefig(VIZ_DIR/"avg_sentiment_daily.png"); plt.show()


## Monthly Scores

In [ ]:

agg=monthly_scores(df)
display(agg.head())


## Top 3 Positive / Negative (latest month)

In [ ]:

tops=top3_by_month(agg)
if len(tops):
    latest=sorted(tops.keys())[-1]
    pos3, neg3 = tops[latest]
    print("Month:", latest)
    print("\nTop 3 Positive:"); display(pos3)
    print("\nTop 3 Negative:"); display(neg3)
else:
    print("No monthly data available.")


## Flight Risk (>=4 negatives in any rolling 30 days)

In [ ]:

risks=flight_risk(df)
print(f"Flight risk employees ({len(risks)}):", risks)


## Linear Regression on Monthly Features

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

X,y=gen_features_for_regression(agg)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
model=LinearRegression().fit(X_train,y_train)
pred=model.predict(X_test)
print("R2:", round(r2_score(y_test,pred),4), " MAE:", round(mean_absolute_error(y_test,pred),4))
print("Coefficients:", dict(zip(X.columns, model.coef_)))


## Save Outputs

In [ ]:

labeled_path = PROJECT_ROOT/"data"/"labeled_output.csv"
agg_path = PROJECT_ROOT/"data"/"monthly_aggregates.csv"
df.to_csv(labeled_path, index=False)
agg.to_csv(agg_path, index=False)
print("Saved:", labeled_path, agg_path)
